In [2]:
import argparse, os, sys, glob
import time
import torchvision.models as models
import datetime
import torch  
import torchvision.transforms as transforms  
from PIL import Image  
from vggloss import *
from omegaconf import OmegaConf
from imwatermark import WatermarkEncoder
import torch
import clip
from PIL import Image
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize
try:
    from torchvision.transforms import InterpolationMode
    BICUBIC = InterpolationMode.BICUBIC
except ImportError:
    BICUBIC = Image.BICUBIC

from matplotlib import pyplot as plt
from ldm.models.diffusion.ddim import DDIMSampler
from ldm.models.diffusion.plms import PLMSSampler
from cocoimagic_utils import *


# environment
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# path
experiment_time_stamp = datetime.date.today()
config_path = "/home/v-penxiao/workspace/DailyExperiments/stable-diffusion/configs/stable-diffusion/v1-inference.yaml"
diffusion_model_path = "/home/v-penxiao/workspace/DailyCkpts/stable_diffusion/v1-5-pruned.ckpt"
# diffusion_model_path = "/home/v-penxiao/workspace/DailyCkpts/stable_diffusion/v1-4-full-ema.ckpt"
img_path = "/home/v-penxiao/workspace/DailyExperiments/stable-diffusion/gray_demo.png"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/flamingo.jpeg"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/bear3.jpeg"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/bird.png"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/cat_3.jpeg"
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray/zebra.jpg"
# img_path = "/home/v-penxiao/workspace/data/imagic_test/gray/landmark_image29231.jpg"

output_path = os.path.join("/home/v-penxiao/workspace/DailyOutputs/stable_outputs/imagic_outputs", str(datetime.date.today()).replace("-","_"))
cond_path = os.path.join(output_path, "opti_cond_xloss.pt")
unet_path = os.path.join(output_path, "opti_unet_xloss.pt")
sample_path = os.path.join(output_path, "samples")
os.makedirs(output_path, exist_ok=True)
os.makedirs(sample_path, exist_ok=True)
base_count = len(os.listdir(sample_path))
grid_count = len(os.listdir(output_path)) - 1
torch.manual_seed(0)

wm = "StableDiffusionV1"
wm_encoder = WatermarkEncoder()
wm_encoder.set_watermark('bytes', wm.encode('utf-8'))
gpu_info()

/anaconda/envs/ldm/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


apex not found
apex not found


'CUDA:0 (NVIDIA A100 80GB PCIe, 80995MiB)'

In [3]:
coco = get_cocosnet().to(device)

config = OmegaConf.load(config_path)
model = load_model_from_config(config, diffusion_model_path)
model = model.to(device)

vgg19_conv2_1_relu = models.vgg19(pretrained=True).features[:7].eval().to(device)
for p in vgg19_conv2_1_relu.parameters():
    p.requires_grad = False
vgg19_conv3_1_relu = models.vgg19(pretrained=True).features[:11].eval().to(device)
for p in vgg19_conv3_1_relu.parameters():
    p.requires_grad = False
clip_model, preprocess = clip.load("ViT-B/32", device=device)
for p in clip_model.parameters():
    p.requires_grad = False
if False:
    sampler = PLMSSampler(model)
else:
    sampler = DDIMSampler(model)
start_code = None

if False:
    start_codenamed_parametersrandn([n_samples, 4, H // 8, W // 8], device=device)
    
precision_scope = autocast if True else nullcontext
batch_size = 1
n_rows = batch_size

nf = 32, f = 32
Network [SPADEGenerator] was created. Total number of parameters: 103.2 million. To see the architecture, do print(network).
Network [NoVGGCorrespondence] was created. Total number of parameters: 50.8 million. To see the architecture, do print(network).
Loading model from /home/v-penxiao/workspace/DailyCkpts/stable_diffusion/v1-5-pruned.ckpt
Global Step: 840000
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels


Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.17.mlp.fc1.bias', 'vision_model.encoder.layers.6.layer_norm2.bias', 'vision_model.encoder.layers.19.self_attn.out_proj.weight', 'vision_model.encoder.layers.6.self_attn.k_proj.bias', 'vision_model.encoder.layers.14.mlp.fc2.bias', 'vision_model.encoder.layers.22.mlp.fc2.bias', 'vision_model.encoder.layers.8.mlp.fc2.weight', 'vision_model.encoder.layers.16.layer_norm2.weight', 'vision_model.encoder.layers.4.layer_norm1.weight', 'vision_model.encoder.layers.20.self_attn.out_proj.weight', 'vision_model.encoder.layers.4.mlp.fc1.bias', 'vision_model.encoder.layers.20.layer_norm2.weight', 'vision_model.encoder.layers.8.layer_norm1.weight', 'vision_model.encoder.layers.14.self_attn.out_proj.bias', 'vision_model.encoder.layers.6.self_attn.v_proj.weight', 'vision_model.pre_layrnorm.bias', 'vision_model.encoder.layers.10.layer_norm2.bias', 'vision_mode

In [4]:
# neg_prompt = "a gray photograph of a dog, a black bench in the center, and the grassland."
neg_prompt = "A grayscale photograph of a bird standing on a branch"
# neg_prompt = "a mostly gray photograph."
# prompt = "a colorful photograph of a yellow dog,  a dark red bench, the bule sky and the green grassland"
prompt = "a yellow dog, sitting on a bench in the grassland"
# prompt = "A colorful photograph of a red flamingo standing by the sea."
# prompt = "A colorful photograph of a black bear walking in the green forest"
# prompt = "A colorful photograph of a bird standing on a branch"
# prompt = "a photograph of a sitting yellow cat"
# prompt = "a colorful photograph of a bowl of red apples"
# prompt = "a big white and red plane flying in the blue sky"
# prompt = "a red bus driving on the street"
# prompt = "a colorful photograph of a brown and white cow lies on a green grass"
# prompt = "a colorful photograph."
# prompt = "A work of art with a purple lion standing on it, a red-haired woman lying on the ground, a yellow violin falling beside her, and a yellow moon in the sky."
# prompt = "A color photograph of houses with colorful roofs and a blue sky in the background"
# prompt = "a colorful photograph of a black horse standing on a path in the countryside with a clear sky in the background."
# resize_transform = transforms.Resize(512)  
# pil_image = Image.open(img_path).convert("RGB")
# resized_image = resize_transform(pil_image)  
src_image = np.array(Image.open(img_path).convert("RGB"))

src_image = src_image.astype(np.float32) / 255.0
src_image = src_image[None].transpose(0, 3, 1, 2)
src_image = torch.from_numpy(src_image)

src_image = src_image * 2.0 - 1.0
src_image = src_image.to(device)

un_cond = model.get_learned_conditioning(batch_size * [""])
cond = model.get_learned_conditioning([prompt])
src_image_latent = model.encode_first_stage(src_image)
x_start = model.get_first_stage_encoding(src_image_latent)

cond.requires_grad = False
cond_copy = cond.clone()
x_start.requires_grad = False

In [6]:
'''
    #3. finetune diffusion model for reconstruction
'''
print("-" * 20, "finetune for reconstruction", "-" * 20)
model.model.train()
cond.requires_grad = False
# optimizer_finetune = torch.optim.Adam([
#                                         {'params':[cond], 'lr':1e-5},
#                                         {'params':model.model.parameters(), 'lr':2e-7}], 
#                                         lr=1e-6)
# optimizer_finetune = torch.optim.Adam([cond], lr=2e-3)
optimizer_finetune = torch.optim.Adam(model.model.parameters(), lr=2e-7)
print("cond.requires_grad=",cond.requires_grad)
print("cond_copy.requires_grad=",cond_copy.requires_grad)
# model.model.eval()
# for p in model.model.parameters():
#     p.requires_grad = False
steps = 3500
criteria = torch.nn.MSELoss()
history = []

# func =  Compose([
#         Resize(256,interpolation=BICUBIC),
#         CenterCrop(224),
#         Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711)),
#     ])

for t_int in range(steps):
    optimizer_finetune.zero_grad()
    
    noise = torch.randn_like(x_start)
    t_tensor = torch.randint(0, model.num_timesteps, (x_start.shape[0],), device=model.device).long()
    print(f"t_tensor = {t_tensor}")
    x_noisy = model.q_sample(x_start, t_tensor, noise=noise)
    
    '''for eps prediction model'''
    pred_noise = model.apply_model(x_noisy, t_tensor, cond)
    pred_x0 = model.predict_start_from_noise(x_noisy, t_tensor, pred_noise)
    '''for x0 prediction model'''
    # pred_x0 = model.apply_model(x_noisy, t_tensor, cond)
    ''''''
    # pred = generate_by_prompt(wm_encoder=wm_encoder, c=cond, samplER=sampler, model=model, output_path=output_path, start_code=start_code,h=512, w=512, ddim_eta=0.0, n_samples=batch_size, scale=3, ddim_steps=45)

    
    pred_x0_rgb = torch.clamp((model.differentiable_decode_first_stage(pred_x0) + 1.0) / 2.0, min=-1.0, max=1.0)
    # coco_x0_rgb = colorized(coco,src_image,pred_x0_rgb, True, output_path)
    
        # print(coco_x0_rgb.max(),coco_x0_rgb.min(),src_image.max(),src_image.min(),pred_x0_rgb.max(),pred_x0_rgb.min)
    x_sample = 255. * rearrange(((1.+pred_x0_rgb[0])/2.).detach().cpu().numpy(), 'c h w -> h w c')
    img = Image.fromarray(x_sample.astype(np.uint8))
    img = put_watermark(img, wm_encoder)
    img.save(os.path.join(sample_path, f"pred_{t_tensor.item()}.png"))
    continue
    # pred_x0_gray = degration(pred_x0_rgb)
    # pred_x0_gray_latent = model.get_first_stage_encoding(model.differentiable_encode_first_stage(pred_x0_gray))

        # loss = criteria(pred_x0_gray_latent, x_start)
    # pixel_loss =criteria(pred_x0_rgb, src_image)
    # vgg_loss = vgg(pred_x0_rgb, src_image)
    # p_loss = vgg19_loss(src_image, pred_x0_rgb, vgg19_conv2_1_relu, vgg19_conv3_1_relu)
    # coco_clip_loss = get_clip_loss(clip_model, coco_x0_rgb, pos_prompt=prompt, neg_prompt=neg_prompt)
    pred_clip_loss = get_clip_loss(clip_model, pred_x0_rgb, pos_prompt=prompt, neg_prompt=neg_prompt)
    # clip_loss = .5 * pred_clip_loss + .5 * coco_clip_loss
    clip_loss = 1. * pred_clip_loss
    noise_loss = criteria(pred_noise, noise)
    # if t_int>1500:
    #     loss = .4 * clip_loss + 0.5 * noise_loss + 0.1 * p_loss
    # else:
    #     loss = .2 * clip_loss + 0.7 * noise_loss + 0.1 * p_loss
    loss = .2 * clip_loss + 0.8 * noise_loss 
    # loss = criteria(pred_x0_rgb, colorized(coco,src_image,pred_x0_rgb))
    loss.backward()
    history.append(loss.item())
    optimizer_finetune.step()
    with torch.no_grad():
        if (t_int+1) % 100 ==0 or t_int ==0:
            print("curr_t=", t_int, "; loss=", loss.item(), f"clip loss = {clip_loss.item()}, noise loss = {noise_loss.item()}")
            ret = generate_by_prompt(wm_encoder=wm_encoder, c=cond, samplER=sampler, model=model, output_path=output_path, start_code=start_code,h=512, w=512, ddim_eta=0.0, n_samples=batch_size, scale=3, ddim_steps=45)
            colored_ret = colorized(coco,src_image,ret.type(src_image.type()).to(src_image.device), True, output_path)
    # x_start =  model.get_first_stage_encoding(model.encode_first_stage(coco_x0_rgb))

# unet_for_pixel_path = "/root/stable/stable-diffusion/outputs/imagic/pixel_unet.pt"

# print("saving finetuned unet-model to path:", unet_for_pixel_path)
# torch.save(model.model.state_dict(), unet_for_pixel_path)
# print("Done\n")

cond_for_pixel_path = os.path.join(output_path,"pixel_cond.pt")

print("saving optimized_cond to path:", cond_for_pixel_path)
# torch.save(new_cond, cond_for_pixel_path)
print("Done\n")

-------------------- finetune for reconstruction --------------------
cond.requires_grad= False
cond_copy.requires_grad= False
t_tensor = tensor([240], device='cuda:0')
t_tensor = tensor([195], device='cuda:0')
t_tensor = tensor([325], device='cuda:0')
t_tensor = tensor([866], device='cuda:0')
t_tensor = tensor([189], device='cuda:0')
t_tensor = tensor([366], device='cuda:0')
t_tensor = tensor([11], device='cuda:0')
t_tensor = tensor([47], device='cuda:0')
t_tensor = tensor([174], device='cuda:0')
t_tensor = tensor([312], device='cuda:0')
t_tensor = tensor([515], device='cuda:0')
t_tensor = tensor([663], device='cuda:0')
t_tensor = tensor([471], device='cuda:0')
t_tensor = tensor([229], device='cuda:0')
t_tensor = tensor([788], device='cuda:0')
t_tensor = tensor([669], device='cuda:0')


KeyboardInterrupt: 

In [1]:
for _ in range(5):
    with torch.no_grad():
        ret = generate_by_prompt(wm_encoder=wm_encoder, c=cond, samplER=sampler, model=model, output_path=output_path, start_code=start_code,h=512, w=512, ddim_eta=0.0, n_samples=batch_size, scale=3, ddim_steps=45)
        colored_ret = colorized(coco,src_image,ret.type(src_image.type()).to(src_image.device), True, output_path)


NameError: name 'torch' is not defined

In [ ]:
'''
    #1. embedding optimization
'''
print("-" * 20, "embedding optimization", "-" * 20)
cond.requires_grad = True
optimizer = torch.optim.Adam([cond], lr=2e-3)
criteria = torch.nn.MSELoss()
history = []
steps = 1000
for t_int in range(steps):
    optimizer.zero_grad()
    
    noise = torch.randn_like(x_start)
    t_tensor = torch.randint(0, model.num_timesteps, (x_start.shape[0],), device=model.device).long()
    x_noisy = model.q_sample(x_start, t_tensor, noise=noise)
    '''for eps prediction model'''
    pred_noise = model.apply_model(x_noisy, t_tensor, cond)
    # pred_x0 = model.predict_start_from_noise(x_noisy, t_tensor, pred_noise)
    '''for x0 prediction model(not available)'''
    # pred_x0 = model.apply_model(x_noisy, t_tensor, cond)
    
    # pred_x0_rgb = torch.clamp((model.differentiable_decode_first_stage(pred_x0) + 1.0) / 2.0, min=-1.0, max=1.0)
    # pred_x0_gray = degration(pred_x0_rgb)
    # pred_x0_gray_latent = model.get_first_stage_encoding(model.differentiable_encode_first_stage(pred_x0_gray))
    
    # save_image(pred_x0_rgb,"pred_rgb.jpg")
    # save_image(pred_x0_gray,"pred_gray.jpg")

    # loss = criteria(pred_x0_gray_latent, x_start)
    # loss = criteria(pred_x0, x_start)
    # loss = criteria(pred_x0_rgb, colorized(coco,src_image,pred_x0_rgb))
    loss = criteria(pred_noise, noise)
    loss.backward()
    history.append(loss.item())
    optimizer.step()
    if (t_int+1) % 100 ==0 or t_int ==0:
        print("curr_t=", t_int, "; loss=", loss.item())
        with torch.no_grad():
            _ = generate_by_prompt(wm_encoder=wm_encoder, c=cond, samplER=sampler, model=model, 
                                   output_path=output_path, start_code=start_code,h=512, w=512, ddim_eta=0.0, 
                                   n_samples=batch_size, scale=3, ddim_steps=45)

print("saving optimized_cond to path:", cond_path)
torch.save(cond, cond_path)
print("Done\n")

In [ ]:
'''
    #2. finetune diffusion model
'''
print("-" * 20, "finetune diffusion model", "-" * 20)
# cond = torch.load(cond_path)
cond.requires_grad = False
print("cond.requires_grad=",cond.requires_grad)
model.model.train()
optimizer_finetune = torch.optim.Adam(model.model.parameters(), lr=5e-7)
steps = 2000
criteria = torch.nn.MSELoss()
history = []

for t_int in range(steps):
    optimizer_finetune.zero_grad()
    
    noise = torch.randn_like(x_start)
    t_tensor = torch.randint(0, model.num_timesteps, (x_start.shape[0],), device=model.device).long()
    x_noisy = model.q_sample(x_start, t_tensor, noise=noise)
    
    '''for eps prediction model'''
    pred_noise = model.apply_model(x_noisy, t_tensor, cond)
    # pred_x0 = model.predict_start_from_noise(x_noisy, t_tensor, pred_noise)
    '''for x0 prediction model'''
    # pred_x0 = model.apply_model(x_noisy, t_tensor, cond)
      
    # pred_x0_rgb = torch.clamp((model.differentiable_decode_first_stage(pred_x0) + 1.0) / 2.0, min=-1.0, max=1.0)
    # pred_x0_gray = degration(pred_x0_rgb)
    # pred_x0_gray_latent = model.get_first_stage_encoding(model.differentiable_encode_first_stage(pred_x0_gray))

    # save_image(pred_x0_rgb,"pred_rgb.jpg")
    # save_image(pred_x0_gray,"pred_gray.jpg")
    
    # loss = 0.0 * criteria(pred_x0_gray_latent, x_start)  + 0. * criteria(pred_noise, noise) + 0. * vgg(pred_x0_gray, src_image)
    # loss = criteria(pred_x0_gray_latent, x_start)
    # loss = criteria(pred_x0, x_start)
    loss = criteria(pred_noise, noise)

    loss.backward()
    history.append(loss.item())
    optimizer_finetune.step()
    
    if (t_int+1) % 100 ==0 or t_int ==0:
        print("curr_t=", t_int, "; loss=", loss.item())
        with torch.no_grad():
            _ = generate_by_prompt(wm_encoder=wm_encoder, c=cond, samplER=sampler, model=model, output_path=output_path, start_code=start_code,h=512, w=512, ddim_eta=0.0, n_samples=batch_size, scale=3, ddim_steps=45)

print("saving finetuned unet-model to path:", unet_path)
torch.save(model.model.state_dict(), unet_path)
print("Done\n")

In [ ]:
cond = torch.load(cond_path)
model.model.load_state_dict(torch.load(unet_path))

In [ ]:
model.model.eval()
cond.requires_grad = False
v = [0.8, 0.85, 0.9, 1., 1.05, 1.1, 1.2, 1.3, 1.4]
n = 3
# heights = [128 for _ in range(n)]
# widths = [128 for _ in range(len(v))]
# fig_width = n * 30  # inches
# fig_height = fig_width * sum(heights) / sum(widths)
# f, axarr = plt.subplots(len(v),n, figsize=(fig_width, fig_height),
#                         gridspec_kw={'height_ratios':heights})
for id, alpha in enumerate(v):
    new_c = cond * torch.Tensor([1-alpha]).to(device) + cond * torch.Tensor([alpha]).to(device)
    for j in range(n):
        with torch.no_grad():
            ret = generate_by_prompt(wm_encoder=wm_encoder, c=new_c, samplER=sampler, model=model, output_path=output_path, start_code=start_code,h=512, w=512, ddim_eta=0.0, n_samples=1, scale=3, ddim_steps=50)

In [ ]:
cond = torch.load(cond_path)
model.model.load_state_dict(torch.load(unet_path))

In [ ]:
'''
    #3. finetune diffusion model for reconstruction
'''
print("-" * 20, "finetune for reconstruction", "-" * 20)
model.model.train()
optimizer_finetune = torch.optim.Adam([
                                        {'params':[cond], 'lr':1e-5},
                                        {'params':model.model.parameters(), 'lr':5e-7}], 
                                        lr=1e-6)
# optimizer_finetune = torch.optim.Adam([cond], lr=2e-3)
# optimizer_finetune = torch.optim.Adam(model.model.parameters(), lr=5e-8)
cond.requires_grad = True
print("cond.requires_grad=",cond.requires_grad)
# model.model.eval()
# for p in model.model.parameters():
#     p.requires_grad = False
steps = 3000
criteria = torch.nn.MSELoss()
history = []

# func =  Compose([
#         Resize(256,interpolation=BICUBIC),
#         CenterCrop(224),
#         Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711)),
#     ])

for t_int in range(steps):
    optimizer_finetune.zero_grad()
    
    noise = torch.randn_like(x_start)
    t_tensor = torch.randint(0, model.num_timesteps, (x_start.shape[0],), device=model.device).long()
    x_noisy = model.q_sample(x_start, t_tensor, noise=noise)
    
    '''for eps prediction model'''
    pred_noise = model.apply_model(x_noisy, t_tensor, cond)
    pred_x0 = model.predict_start_from_noise(x_noisy, t_tensor, pred_noise)
    '''for x0 prediction model'''
    # pred_x0 = model.apply_model(x_noisy, t_tensor, cond)
    
    pred_x0_rgb = torch.clamp((model.differentiable_decode_first_stage(pred_x0) + 1.0) / 2.0, min=-1.0, max=1.0)
    coco_x0_rgb = colorized(coco,src_image,pred_x0_rgb, False, output_path)
    
        # print(coco_x0_rgb.max(),coco_x0_rgb.min(),src_image.max(),src_image.min(),pred_x0_rgb.max(),pred_x0_rgb.min)
        # x_sample = 255. * rearrange(((1.+pred_x0_rgb[0])/2.).detach().cpu().numpy(), 'c h w -> h w c')
        # img = Image.fromarray(x_sample.astype(np.uint8))
        # img = put_watermark(img, wm_encoder)
        # img.save(os.path.join(sample_path, f"coco_{t_int}.png"))
        # pred_x0_gray = degration(pred_x0_rgb)
        # pred_x0_gray_latent = model.get_first_stage_encoding(model.differentiable_encode_first_stage(pred_x0_gray))

        # loss = criteria(pred_x0_gray_latent, x_start)
    # pixel_loss =criteria(pred_x0_rgb, src_image)
    # vgg_loss = vgg(pred_x0_rgb, src_image)
    # p_loss = vgg19_loss(src_image, pred_x0_rgb, vgg19_conv2_1_relu,vgg19_conv3_1_relu)
    coco_clip_loss = get_clip_loss(clip_model, coco_x0_rgb, pos_prompt=prompt, neg_prompt=neg_prompt)
    pred_clip_loss = get_clip_loss(clip_model, pred_x0_rgb, pos_prompt=prompt, neg_prompt=neg_prompt)
    clip_loss = 1.0 * coco_clip_loss + 1. * pred_clip_loss
    noise_loss = criteria(pred_noise, noise)
    if t_int>2500:
        loss = 1 * clip_loss + 0.1 * noise_loss 
    else:
        loss = 0.2 * clip_loss + 1. * noise_loss 
    # loss = criteria(pred_x0_rgb, colorized(coco,src_image,pred_x0_rgb))
    loss.backward()
    history.append(loss.item())
    optimizer_finetune.step()
    if (t_int+1) % 100 ==0 or t_int ==0:
        print("curr_t=", t_int, "; loss=", loss.item(), f"clip loss = {clip_loss.item()}, noise loss = {noise_loss.item()}")
        # new_cond = new_cond * torch.Tensor([1-0.1]).to(device) + model.get_learned_conditioning([prompt]) * torch.Tensor([0.1]).to(device)
        ret = generate_by_prompt(wm_encoder=wm_encoder, c=cond, samplER=sampler, model=model, output_path=output_path, start_code=start_code,h=512, w=512, ddim_eta=0.0, n_samples=batch_size, scale=3, ddim_steps=45)
        colored_ret = colorized(coco,src_image,ret.type(src_image.type()).to(src_image.device), True, output_path)

# unet_for_pixel_path = "/root/stable/stable-diffusion/outputs/imagic/pixel_unet.pt"

# print("saving finetuned unet-model to path:", unet_for_pixel_path)
# torch.save(model.model.state_dict(), unet_for_pixel_path)
# print("Done\n")

cond_for_pixel_path = os.path.join(output_path,"pixel_cond.pt")

print("saving optimized_cond to path:", cond_for_pixel_path)
# torch.save(new_cond, cond_for_pixel_path)
print("Done\n")

In [ ]:
src_image = np.array(Image.open("/home/v-penxiao/workspace/DailyExperiments/stable-diffusion/data/inpainting_examples/overture-creations-5sI6fQgYIuo.png").convert("RGB"))
src_image = src_image.astype(np.float32) / 255.0
src_image = src_image[None].transpose(0, 3, 1, 2)
src_image = torch.from_numpy(src_image)
src_image = src_image * 2.0 - 1.0
src_image = src_image.to(device)
import torch
import clip
from PIL import Image
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize
try:
    from torchvision.transforms import InterpolationMode
    BICUBIC = InterpolationMode.BICUBIC
except ImportError:
    BICUBIC = Image.BICUBIC
    
func =  Compose([
        Resize(256,interpolation=BICUBIC),
        CenterCrop(224),
        Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711)),
    ])

# image = preprocess(Image.open(img_path)).unsqueeze(0).to(device)
src_image = (src_image+1.)/2.
image = func(src_image)
image.shape

In [ ]:
import torch
import clip
from PIL import Image
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize
try:
    from torchvision.transforms import InterpolationMode
    BICUBIC = InterpolationMode.BICUBIC
except ImportError:
    BICUBIC = Image.BICUBIC
    
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model, preprocess = clip.load("ViT-B/32", device=device)
neg_prompt = "a grayscale photo"
# neg_prompt = "a mostly gray photograph."
prompt = "a colorful photo"
# prompt = "a colorful photograph."
# image = preprocess(Image.open("/home/v-penxiao/workspace/DailyExperiments/stable-diffusion/data/inpainting_examples/overture-creations-5sI6fQgYIuo.png")).unsqueeze(0).to(device)
#/home/v-penxiao/workspace/DailyOutputs/stable_outputs/imagic_outputs/gray_demo.png
image = preprocess(Image.open("/home/v-penxiao/workspace/DailyOutputs/stable_outputs/imagic_outputs/2023_04_03/samples/00252.png")).unsqueeze(0).to(device)
i_e = clip_model.encode_image(image)
p_t_e = clip_model.encode_text(clip.tokenize(prompt).to(image.device))
n_t_e = clip_model.encode_text(clip.tokenize(neg_prompt).to(image.device))
g_t_e = clip_model.encode_text(clip.tokenize("grayscale").to(image.device))
# gray_t_e = model.encode_text(clip.tokenize("a gray photograph").to(image.device))
# color_t_e = model.encode_text(clip.tokenize("a colorful photograph").to(image.device))
p_cos_sim = torch.nn.functional.cosine_similarity(i_e, p_t_e).mean()
n_cos_sim = torch.nn.functional.cosine_similarity(i_e, n_t_e).mean()
g_cos_sim = torch.nn.functional.cosine_similarity(i_e, g_t_e).mean()
print(p_cos_sim)
print(n_cos_sim)
print(g_cos_sim)

In [ ]:
n_cos_sim

In [ ]:
torch.nn.functional.cosine_similarity(i_e, n_t_e).mean()

In [ ]:
torch.nn.functional.cosine_similarity(i_e, p_t_e).mean()

In [ ]:
clip_model.encode_text(clip.tokenize([prompt,neg_prompt]).to(device)).shape

In [ ]:
cond.shape

In [ ]:
clip.tokenize(neg_prompt).shape

In [ ]:
from PIL import Image
import os

directory_in_str = '/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests'
output_directory = '/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests_gray'

directory = os.fsencode(directory_in_str)

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith('.png') or filename.endswith('.jpg') or filename.endswith('.jpeg'):
        filepath = os.path.join(directory_in_str, filename)
        new_filepath = os.path.join(output_directory, filename)
        im = Image.open(filepath).convert('L').resize((512, 512))
        im.save(new_filepath)

In [ ]:
import numpy as np
from PIL import Image
import torch
src_image = np.array(Image.open("/home/v-penxiao/workspace/DailyOutputs/stable_outputs/imagic_outputs/gray_demo.png").convert("RGB"))
src_image = src_image.astype(np.float32) / 255.0
src_image = torch.from_numpy(src_image).permute(2,0,1)


In [ ]:
src_image.shape

In [ ]:
from segment_anything import SamPredictor, sam_model_registry
sam = sam_model_registry["default"](checkpoint="/home/v-penxiao/workspace/DailyCkpts/sam/sam_vit_h_4b8939.pth")
predictor = SamPredictor(sam)
predictor.set_image(src_image)
masks, _, _ = predictor.predict("dog")

In [ ]:
from segment_anything import SamAutomaticMaskGenerator, sam_model_registry
mask_generator = SamAutomaticMaskGenerator(sam)
masks = mask_generator.generate(src_image)

In [ ]:
import numpy as np
import cv2
import os
import torch
import math
import torchvision.transforms as transforms
from PIL import Image


# 单独设置
# 随机改变图像的亮度
brightness_change = transforms.ColorJitter(brightness=0.1)
# 随机改变图像的色调
hue_change = transforms.ColorJitter(hue=0.1)
# 随机改变图像的对比度
contrast_change = transforms.ColorJitter(contrast=0.1)

# 综合设置
color_aug = transforms.ColorJitter(brightness=0.3, contrast=0.5, saturation=0.1, hue=0.3)

transform = transforms.Compose([
        # brightness_change,
        # hue_change,
        # contrast_change,
        color_aug
    ])
# img_path = "/home/v-penxiao/workspace/DailyOutputs/stable_outputs/tests/apples.jpeg"
img_path = "/home/v-penxiao/workspace/cocosnet4imagic/CoCosNet/before.jpg"
src_image = cv2.imread(img_path)
color = cv2.cvtColor(src_image, cv2.COLOR_BGR2RGB)
color = Image.fromarray(color)
color = transform(color)
color = np.asarray(color)
cv2.imwrite("yy.jpg", color)
# color = cv.cvtColor(color, cv.COLOR_RGB2BGR)
# save_image(src_image, "yy.jpg")
# save_image(color, "xx.jpg")

In [ ]:
func = LineCollator(img_size=src_image.shape[1])


In [ ]:
from torchvision.transforms import ColorJitter
import cv2 as cv


def uniform_grid(shape):
    '''Uniform grid coordinates.
    
    Params
    ------
    shape : tuple
        HxW defining the number of height and width dimension of the grid
    Returns
    -------
    points: HxWx2 tensor
        Grid coordinates over [0,1] normalized image range.
    '''

    H, W = shape[:2]
    c = np.empty((H, W, 2))
    c[..., 0] = np.linspace(0, 1, W, dtype=np.float32)
    c[..., 1] = np.expand_dims(np.linspace(0, 1, H, dtype=np.float32), -1)

    return c


def tps_theta_from_points(c_src, c_dst, reduced=False):
    delta = c_src - c_dst
    
    cx = np.column_stack((c_dst, delta[:, 0]))
    cy = np.column_stack((c_dst, delta[:, 1]))
        
    theta_dx = TPS.fit(cx, reduced=reduced)
    theta_dy = TPS.fit(cy, reduced=reduced)

    return np.stack((theta_dx, theta_dy), -1)


def tps_grid(theta, c_dst, dshape):
    ugrid = uniform_grid(dshape)

    reduced = c_dst.shape[0] + 2 == theta.shape[0]

    dx = TPS.z(ugrid.reshape((-1, 2)), c_dst, theta[:, 0]).reshape(dshape[:2])
    dy = TPS.z(ugrid.reshape((-1, 2)), c_dst, theta[:, 1]).reshape(dshape[:2])
    dgrid = np.stack((dx, dy), -1)

    grid = dgrid + ugrid
    
    return grid # H'xW'x2 grid[i,j] in range [0..1]


def tps_grid_to_remap(grid, sshape):
    '''Convert a dense grid to OpenCV's remap compatible maps.
    
    Params
    ------
    grid : HxWx2 array
        Normalized flow field coordinates as computed by compute_densegrid.
    sshape : tuple
        Height and width of source image in pixels.
    Returns
    -------
    mapx : HxW array
    mapy : HxW array
    '''

    mx = (grid[:, :, 0] * sshape[1]).astype(np.float32)
    my = (grid[:, :, 1] * sshape[0]).astype(np.float32)

    return mx, my


def warping_image(img, c_src, c_dst):
    dshape = img.shape
    theta = tps_theta_from_points(c_src, c_dst, reduced=True)
    grid = tps_grid(theta, c_dst, dshape)
    mapx, mapy = tps_grid_to_remap(grid, img.shape)
    
    return cv.remap(img, mapx, mapy, cv.INTER_CUBIC)

class LineCollator:
    """Collator for training.
    """
    def __init__(self,
                 img_size=224,
                 src_perturbation=0.5,
                 dst_perturbation=0.2,
                 brightness=0.3,
                 contrast=0.5,
                 saturation=0.1,
                 hue=0.3):

        self.size = img_size
        self.src_per = src_perturbation
        self.tgt_per = dst_perturbation
        self.thre = 50

        self.src_const = np.array([
            [-0.5, -0.5],
            [0.5, -0.5],
            [-0.5, 0.5],
            [0.5, 0.5],
            [0.2, -0.2],
            [-0.2, 0.2],
            [0.2, 0.2],
            [-0.2, -0.2]
        ])

        self.jittering = ColorJitter(brightness, contrast, saturation, hue)

    @staticmethod
    def _random_crop(line, color, size):
        height, width = line.shape[0], line.shape[1]
        rnd0 = np.random.randint(height - size - 1)
        rnd1 = np.random.randint(width - size - 1)

        line = line[rnd0: rnd0 + size, rnd1: rnd1 + size]
        color = color[rnd0: rnd0 + size, rnd1: rnd1 + size]

        return line, color

    @staticmethod
    def _coordinate(image):
        """3 stage of manipulation
           - BGR -> RGB
           - (H, W, C) -> (C, H, W)
           - Normalize
        
        Parameters
        ----------
        image : numpy.array
            image data
        
        Returns
        -------
        numpy.array
            manipulated image data
        """
        image = image[:, :, ::-1]
        image = image.transpose(2, 0, 1)
        image = (image - 127.5) / 127.5

        return image

    @staticmethod
    def _totensor(array_list):
        array = np.array(array_list).astype(np.float32)
        tensor = torch.FloatTensor(array)
        tensor = tensor.cuda()

        return tensor

    def _warp(self, img):
        """Spatial augment by TPS
        """
        const = self.src_const
        c_src = const + np.random.uniform(-self.src_per, self.src_per, (8, 2))
        c_tgt = c_src + np.random.uniform(-self.tgt_per, self.tgt_per, (8, 2))

        img = warping_image(img, c_src, c_tgt)

        return img

    def _jitter(self, img):
        """Color augment
        """
        img = img.astype(np.float32)
        noise = np.random.uniform(-self.thre, self.thre)
        img += noise
        img = np.clip(img, 0, 255)

        return img

    def _prepair(self, color, line):
        """3 stages of preparation
           - Crop
           - Spatial & Color augmentation
           - Coordination
        """
        line, color = self._random_crop(line, color, size=self.size)

        jittered = self._jitter(color)
        warped = self._warp(jittered)

        jittered = self._coordinate(jittered)
        warped = self._coordinate(warped)
        line = self._coordinate(line)

        return jittered, warped, line

    def __call__(self, batch):
        j_box = []
        w_box = []
        l_box = []

        for b in batch:
            color, line = b
            jitter, warped, line = self._prepair(color, line)

            j_box.append(jitter)
            w_box.append(warped)
            l_box.append(line)

        j = self._totensor(j_box)
        w = self._totensor(w_box)
        l = self._totensor(l_box)

        return (j, w, l)